2022/07/02

Processing to format Natural Questions train set. Test set can not be used as no answers given. 

Validation set will be split through train_test_split after tokenization. 

Format: "question": question, "context": long_answer_str, "answers": short_targets,  "category": category, 'id': id

Output data path: "drive/MyDrive/w266/data/natural_questions/data_v2.json"

### Install dependences

In [ ]:
### Import Libraries and Dependencies
!pip install transformers
!pip install datasets
!pip install apache_beam
!pip install git+https://github.com/huggingface/transformers.git
!pip install huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.3 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 10.3 MB/s 
     |████████████████████████████████| 140 kB 14.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.0 MB/s 
     |████████████████████████████████| 212 kB 66.9 MB/s 
     |████████████████████████████████| 127 kB 75.3 MB/s 
     |████████████████████████████████| 271 kB 61.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |███████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-zvasrd91
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-zvasrd91
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4578979 sha256=4b172cd2567e89cd10f95e9d65bc9cd30f99c384ca903b0abf428434f7c28f89
  Stored in directory: /tmp/pip-ephem-wheel-cache-zwgh0tog/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [ ]:
from datasets import load_dataset
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
import transformers
import pyarrow as pa
from datasets import load_from_disk
from transformers import AutoTokenizer, TFAutoModel
import jax
import jax.numpy as jnp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Data

In [ ]:
!ls drive/MyDrive/w266/data/natural_questions/

processed_train.json	  simplified-nq-train.jsonl  train_set.json
simplified-nq-test.jsonl  test_set.json		     wlong_nq_train.json


In [ ]:
fn = '/content/drive/MyDrive/w266/data/natural_questions/wlong_nq_train.json'
data = load_from_disk(fn)

### Preprocess

In [ ]:
sample=data[0]

In [ ]:
sample.keys()

dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])

In [ ]:
# require tokenizer 
context_str = sample['document_text']

In [ ]:
def format_dataset(sample):
  question = sample['question_text']
  
  # is_html = sample['document']['tokens']['is_html']
  # long_answers = sample['annotations'][0]['long_answer']
  id = sample['example_id']

  # long answer process; some redundant code, double check
  context_str = sample['document_text'] 
  long_start_token = sample['annotations'][0]['long_answer']['start_token'] 
  long_end_token = sample['annotations'][0]['long_answer']['end_token']
  long_answer_token = context_str.split(' ')[long_start_token:long_end_token]
  long_answer_str = ' '.join(long_answer_token)

  # change token index to corrsponding indexs in the long answer instead of in context
  short_answers = sample['annotations'][0]['short_answers']
  short_targets = {}
  short_targets['answer_start'] = []
  short_targets['text'] = []
  for j in range(len(short_answers)):
    short_start_token = short_answers[j]['start_token'] 
    short_end_token = short_answers[j]['end_token'] 
    # corresponding short answer indexes in the long-answer text
    c_start_token = short_start_token - long_start_token
    c_end_token = short_end_token - long_start_token

    short_targets['answer_start'].append(c_start_token)
    # short_targets['end_token'].append(c_end_token)
    short_answer_text = ' '.join(long_answer_token[c_start_token:c_end_token])
    short_targets['text'].append(short_answer_text)
  
  #yes or no 
  category = int()
  answer = sample['annotations'][0]['yes_no_answer']
  if answer == 'YES':
    category = 1 
  elif answer == 'NO':
    category = 0
  elif answer == 'NONE':
    category = 2

  # short_targets = []
  # for s in short_answers:
  #   short_targets.extend(s['text'])
  # short_targets = list(set(short_targets)). ## this should be texts

  return {"question": question, "context": long_answer_str, "answers": short_targets,  "category": category, 'id': id}

In [ ]:
preprocessed_data = data.map(format_dataset).remove_columns(['annotations', 'question_text','document_url','long_answer_candidates','document_text'])

  0%|          | 0/152148 [00:00<?, ?ex/s]

In [ ]:
# match squad 2 annotations
preprocessed_data.save_to_disk("drive/MyDrive/w266/data/natural_questions/data_v2.json")